In [2]:
import os
import json
import re
import time
import requests, psycopg2
import traceback
from bs4 import BeautifulSoup

In [52]:
visitted_link = []
visitted_title = []
visitted_paragraph = []

conn = psycopg2.connect(
    host="34.124.182.95",
    database="article_data",
    user="ai_user",
    password="{{PASSWORD}}"
)

cur = conn.cursor()

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    
    cleantext = cleantext.replace('\n', '')
    cleantext = cleantext.replace('\xa0', ' ')
    
    return cleantext.strip()


def crawl(visit_link, keyword):
    if visit_link in visitted_link:
        return 0
    else:
        visitted_link.append(visit_link)
    
    print("visit", visit_link)

    data = requests.get(visit_link)
    soup = BeautifulSoup(data.text)
    
    for link in soup.find_all('a'):
        print(len(visitted_title),"LENGTH visitted_title")
        
        link_str = str(link.get('href'))
        
        if link_str.startswith("/node"):
            link_str = link_str[:link_str.find("#")]
            link_str = "https://www.blognone.com" + link_str
        else:
            continue

        if keyword not in link_str:
            continue

        if not link_str.startswith("http"):
            continue
        
        response = requests.get(link_str)
        soup = BeautifulSoup(response.content, 'html.parser')

        
        if link_str != 'https://www.blognone.com/' and 'jobs.blognone.com' not in link_str:
            title = soup.title.get_text().split('|')[0]

            content = ""
            paragraph = []

            
            field_items = soup.find_all('div', {'class': 'field-item'})

            p_tags = []
            for field_item in field_items:
                p_tags.extend(field_item.find_all('p'))

            # Extract the text content of each p tag
            for p_tag in p_tags:
                if (p_tag.get_text()).startswith("ที่มา"):
                    break
                if p_tag.get_text() != "":
                    paragraph.append(p_tag.get_text())


            if title not in visitted_title:

#                 print("WRITE", title + ",,," + str(paragraph) + "\n")
#                 f = open("test.txt", "a")
#                 f.write(link_str + ",,," + title + ",,," + str(paragraph) + "\n")
#                 f.close()
                
                print("import db")
                try:
                    cur.execute("BEGIN")
                    cur.execute(
                        "INSERT INTO article_raw_data (url, category, title, content, paragraphs, raw_html, raw_json, remark) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
                        (link_str, "Technology", title, "", json.dumps(paragraph, ensure_ascii=False), str(soup), "{}", "Peng test spin9 demo (out of scope in sheet), able to delete anytimes")
                    )
                    cur.execute("COMMIT")
                except psycopg2.Error as e:
                    conn.rollback()
                    print("Error: {}".format(e))

                    
                visitted_link.append(link_str)
                visitted_title.append(title)
            else:
                print("link already visitted")
                
        crawl(link_str, keyword)


            
root_link = "https://www.blognone.com/"
data = []
crawl(root_link, keyword="blognone")

conn.commit()
cur.close()
conn.close()




visit https://www.blognone.com/
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
0 LENGTH visitted_title
WRITE Workshop on Trend, Technology, and collaborative opportunities in HPC ,,,['โครงการไทยกริดแห่งชาติร่วมมือกับทาง สวทช. และ สกอ. และนักวิจัยชั้นนำจากสหรัฐร่วมมือ\nจัดงานวิชาการ Workshop on Trend, Technology and collaborative opportunities in High Performance and Grid Computing\nณ ห้องประชุมธีระสูตบุตร อาคารสารสนเทศ 50 ปี มหาวิทยาลัยเกษตรศาสตร์\nวันที่ 23 พฤษภาคม พ.ศ. 2549 เวลา 8.30 - 15.30 น. การสัมมนาไม่เสียค่าใช้จ่ายใดๆ ทั้งสิ้น\nลงทะเบียนกรุณาติดต่อ คุณภัครจิรา ใจยิ้ม โทร 0 2942 8555 ต่อ 1416']

1 LENGTH visitted_title
WRITE มาดูการจัดอันดับมือถือดี/แย่สุดจาก "การวัดรังสีที่ปล่อยออกมา" ,,,['มาดูการจัดอันดับโทรศัพท์มือถือดี/แย่สุดจาก "การวัดรังสีที่ปล่อยออกมา" กัน ออกจากแปลกซักหน่อยเทียบกับการจัดอันดับที่เคยเห็นกัน แต่ผมว่าเข้าท่าทีเดียว

link already visitted
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
6 LENGTH visitted_title
link already visitted
6 LENGTH visitted_title
WRITE Apple Pay เปิดตัวในเกาหลีใต้ เหลือไทยรายเดียวในเอเชียที่มี Apple Store แต่ไม่มี Apple Pay ,,,['Apple ประกาศเตรียมเปิดให้บริการ Apple Pay ในเกาหลีใต้ พร้อมๆ กับการเปิดตัว iPhone 14 สีเหลือง โดยยังไม่ระบุวันเปิดให้บริการ', 'การเปิดบริการ Apple Pay ในเกาหลีใต้ ทำให้ตอนนี้ประเทศไทยเป็นประเทศเดียวในเอเชีย ที่มี Apple Store แต่ยังไม่มีบริการ Apple Pay โดยประเทศอื่นนอกจากเกาหลีใต้ก็มีญี่ปุ่น, จีน, ไต้หวัน, ฮ่องกง, มาเก๊าและสิิงคโปร์']

7 LENGTH visitted_title
7 LENGTH visitted_title
7 LENGTH visitted_title
link already visitted
7 LENGTH visitted_title
7 LENGTH visitted_title
7 LENGTH visitted_title
7 LENGTH visitted_title
7 LENGTH visitted_title
link already visitted
7 LENGTH visitted_title
WRITE YouTube จะย

link already visitted
13 LENGTH visitted_title
WRITE กลุ่ม Sea รายงานผลประกอบการไตรมาส 4/2022 พลิกมามีกำไรสุทธิ 422.8 ล้านดอลลาร์ ,,,['กลุ่ม Sea Limited รายงานผลประกอบการประจำไตรมาสที่ 4 ปี 2022 รายได้รวมตามบัญชี GAAP 3,451.6 ล้านดอลลาร์ เพิ่มขึ้น 7.1% เมื่อเทียบกับไตรมาสเดียวกันในปี 2021 และไตรมาสนี้บริษัทมีกำไรเป็นบวกอยู่ที่ 422.8 ล้านดอลลาร์ จากที่ไตรมาสก่อนหน้าขาดทุน', 'รายได้ของกลุ่มอีคอมเมิร์ซ (Shopee) เพิ่มขึ้น 31.8% เป็น 2,231.1 ล้านดอลลาร์ จำนวนคำสั่งซื้อ 1.7 พันล้านคำสั่ง เทียบกับปีก่อนที่ 2.0 พันล้านคำสั่ง มีมูลค่าการขายสุทธิ 18.0 พันล้านดอลลาร์ เทียบกับปีก่อนที่ 18.2 พันล้านดอลลาร์ ภาพรวมของตลาดเอเชียมี EBITDA เป็นบวกที่ 320.0 ล้านดอลลาร์ ส่วนตลาดอื่นที่เหลือรวมกัน EBITDA เป็นลบ 123.9 ล้านดอลลาร์', 'กลุ่มสื่อบันเทิง (Garena) รายได้ 948.9 ล้านดอลลาร์ ลดลงจากปีก่อน แต่เพิ่มขึ้นจากไตรมาสก่อนหน้า มีจำนวนผู้เล่น 485.5 ล้านบัญชี เป็นผู้เล่นที่จ่ายเงิน 43.6 ล้านบัญชี และกลุ่มการเงินดิจิทัล (Sea Money) มีรายได้ 380.2 ล้านดอลลาร์ และมี EBITDA เป็นบวกที่ 75.6 ล้านดอลลาร์', 'Forrest L

WRITE Blognone ,,,['Google เปิดบริการ VPN แถมไปกับผู้จ่ายค่าใช้งาน Google One แพ็กเกจ 2TB มาตั้งแต่ปี 2020 แต่ล่าสุดก็เตรียมแถมให้กับทุกคน รวมถึงคนใช้แพ็กเกจ 1.99 ดอลลาร์ต่อเดือน อย่างไรก็ดีบริการ VPN นั้นให้บริการบางประเทศเท่านั้น และที่ผ่านมาก็ไม่เปิดบริการในประเทศไทย', 'นอกจากนี้กูเกิลยังประกาศเพิ่มบริการสแกนข้อมูลที่หลุดดออกไปยัง dark web โดยผู้ใช้ต้องใส่ข้อมูลต่างๆ ใ้หกับกูเกิล เช่น ชื่อ, ที่อยู่, หมายเลขโทรศัพท์, เลขประจำตัวประชาชน แล้วกูเกิลจะคอยตรวจสอบว่ามีฐานข้อมูลหลุดใดมีข้อมูลเหล่านี้หรือไม่ หากพบก็จะแจ้งเตือนพร้อมกับแนะนำว่าควรทำอะไรบ้าง', 'Azure Quantum เปิดฟีเจอร์ Integrated Hybrid เปิดทางให้ลูกค้ารันอัลกอริธึมผสมระหว่างคอมพิวเตอร์ปกติและคอมพิวเตอร์ควอนตัม ไมโครซอฟท์สาธิตฟีเจอร์นี้ด้วยโค้ดภาษา Q# ทดสอบคุณสมบัติ entanglement จำนวน 10 รอบ หากระบบทำงานได้ถูกต้องก็จะตอบค่า 0 เสมอ แต่ในความเป็นจริงเมื่อรันไปหลายๆ รอบก็จะได้ค่าที่ผิดไปออกมาบ้าง', 'ตอนนี้คอมพิวเตอร์ควอนตัมที่รองรับฟีเจอร์ Integrated Hybrid ยังใช้ได้กับ Quantinuum H-Series เท่านั้นทางไมโครซอฟท์ระบุว่าจะรองรับคอมพ

link already visitted
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
link already visitted
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
24 LENGTH visitted_title
link already visitted
24 LENGTH visitted_title
WRITE AMD ยืนยันชิป Exynos รุ่นเรือธงของ Samsung มาพร้อมจีพียู RDNA 2  ,,,['ในงาน Keynote ของ AMD ในงาน Computex วันนี้ Lisa Su ซีอีโอของ AMD ออกมายืนยันเองว่าชิป Exynos เรือธงของ Samsung จะมาพร้อมกับจีพียู RDNA 2 ของ AMD เป็นครั้งแรกบนมือถือ และจะมีฟีเจอร์ ray tracing ช่วยให้แสงเงาสมจริง, variable rate shading ปรับอัตรา shading ต่างกันในแต่ละส่วนของมุมมองเพื่อลดการกินสเปกเครื่อง พร้อมระบุว่า Samsung จะให้ข้อมูลเพิ่มเติมเกี่ยวกับชิป Exynos ตัวใหม่ภายในปีนี้ (แต่ไม่ได้ระบุว่าชิปตัวนี้จะออกภายในปีนี้ด้วยหรือไม่)', 'Exynos เป็นชิปมือถือที่ Samsung ผลิตเอง เพื่อวางขายแค่ในบางตลาด และถูกวิจารณ์บ่อยครั้งว่าท

...

link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_

396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=382&p=
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_t

link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
li

link already visitted
visit https://www.blognone.com//node?page=4&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=5&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=6&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=7&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=8&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
visit https://www.blognone.com//node?page=3790&choice=2&form_id=poll_view_voting&op=vot
396 LENGTH visitted_title
396 LENGTH visitted_t

link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link already visitted
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
396 LENGTH visitted_title
link

In [53]:
len(visitted_link)

498

In [24]:
import requests
from bs4 import BeautifulSoup

def crawl(url):
    # Get the HTML content of the page
    response = requests.get(url)

    # Create a Beautiful Soup object and parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the div tags with class field-item
    field_items = soup.find_all('div', {'class': 'field-item'})

    # Extract the p tags from each field-item
    p_tags = []
    for field_item in field_items:
        p_tags.extend(field_item.find_all('p'))

    # Extract the text content of each p tag
    p_texts = []
    for p_tag in p_tags:
        p_texts.append(p_tag.get_text())

    return p_texts

# Example usage
root_url = "https://www.blognone.com/node/132927"
p_texts = crawl(root_url)
for p in p_texts:
    print("@@@", p)


@@@ TikTok เปิดตัว TikTok Series ระบบคอนเทนต์แบบพรีเมียม ที่ต้องจ่ายเงินเพื่อดูคอนเทนต์ โดยที่เรียกเป็น Series เพราะคอนเทนต์เสียเงินจะถูกจัดกลุ่มเป็นคอลเล็คชัน ในหนึ่ง Series จะมีคลิปได้สูงสุด 80 คลิป แต่ละคลิปความยาวสูงสุด 20 นาที
@@@ ราคาที่สามารถตั้งได้ก็มีตั้งแต่ 1 เหรียญไปจนถึง 190 เหรียญ เบื้องต้นระบบนี้จะเปิดใช้กับครีเอเตอร์บางคนก่อน โดยช่วงแรก TikTok ยังไม่หักค่าธรรมเนียม มีแค่หักค่าธุรกรรมธนาคารและค่าธรรมเนียมของ App Store หรือ Play Store
@@@ แม้ระบบ paywall ของ TikTok จะมาทรงเดียวกับ Patreon หรือ OnlyFans แต่นโยบายของ TikTok ยังคงห้ามเรื่องคอนเทนต์โป๊เปลือยอยู่เหมือนเดิม
@@@ ที่มา - TikTok
@@@ 
@@@ ย่อหน้าสุดท้ายดับฝันเลย
@@@ แอพจีนน่าจะยากเรื่อง 18+
@@@ จริง ๆ ตอนนี้ ig ก็ใกล้เคียงแล้วนะครับ 😅


In [1]:
f = open("blognone_url.txt","r")
for line in f:
    url_list = eval(line)

len(url_list)

77029

In [ ]:
conn = psycopg2.connect(
    host="34.124.182.95",
    database="article_data",
    user="ai_user",
    password="{{PASSWORD}}"
)

cur = conn.cursor()
cur.execute("SELECT url from article_raw_data")
result = cur.fetchall()

inserted_list = []
for r in result:
    inserted_list.append(r[0])


for url in sorted(list(url_list))[1:]:

    
    if url in inserted_list:
        continue
    if '#' in url or '?' in url:
        continue

    print(url)
    try:
        data = requests.get("https://www.blognone.com"+url)
    except:
        data = requests.get("https://www.blognone.com"+url)
    soup = BeautifulSoup(data.text)
    content_soup = soup.find_all("div", {"class": "entry-content"})

    time.sleep(0.1)
    
    title = soup.title.get_text()

    p_tags = []
    for content in content_soup:
        p_tags.extend(content.find_all('p'))

    paragraph = []
    for p_tag in p_tags:
        if (p_tag.get_text()).startswith("ที่มา"):
            break
        if p_tag.get_text() != "":
            paragraph.append(p_tag.get_text())

    try:
        cur.execute("BEGIN")
        cur.execute(
            "INSERT INTO article_raw_data (url, category, title, content, paragraphs, raw_html, raw_json, remark) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
            (url, "Marketing", title, "", json.dumps(paragraph, ensure_ascii=False), str(soup), "{}", "crawl page 1-1143")
        )
        cur.execute("COMMIT")
    except psycopg2.Error as e:
        conn.rollback()
        print("Error: {}".format(e))
        
    inserted_list.append(url)

In [5]:
url = "https://www.blognone.com"+sorted(list(url_list))[0]

print(url)

# data = requests.get(url)
# soup = BeautifulSoup(data.text)

# content_soup = soup.find_all("div", {"class": "entry-content"})


https://www.blognone.com/node
